In [ ]:
from math import ceil

import torch
import torch.nn.functional as F
from torch.nn import Linear
import time

from torch_geometric.loader import DataLoader
from torch_geometric.nn import DenseGraphConv, GCNConv, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch

import sys

sys.path.append("../")
import utils

import torch

import torch
import torch.nn.functional as F

from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [ ]:
start = time.time()
train_dataset = utils.GraphDataset("../data/", "GunPoint", n_quantiles=40)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = utils.GraphDataset("../data/", "GunPoint", False, n_quantiles=40)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))
for elem in train_dataset:
    break
elem

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=16, first_pool=30):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels, improved=True)
        self.pool1 = Linear(hidden_channels, first_pool)

        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        self.pool2 = Linear(hidden_channels, ceil(0.5 * first_pool))

        self.conv3 = DenseGraphConv(hidden_channels, hidden_channels)

        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, batch, edge_attr = (
            data.x,
            data.edge_index,
            data.batch,
            data.edge_attr,
        )
        x = self.conv1(x, edge_index, edge_attr).relu()

        x, mask = to_dense_batch(x, batch)
        adj = torch.squeeze(to_dense_adj(edge_index, batch, edge_attr))

        s = self.pool1(x)
        x, adj, mc1, o1 = dense_mincut_pool(x, adj, s, mask)

        x = self.conv2(x, adj).relu()

        s = self.pool2(x)
        x, adj, mc2, o2 = dense_mincut_pool(x, adj, s)

        x = self.conv3(x, adj)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), mc1 + mc2, o1 + o2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(4, 2, hidden_channels=32, first_pool=25).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, patience=5, mode="min", cooldown=2, factor=0.1, verbose=True
# )

In [ ]:
def train(epoch):
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        y_out, mc_loss, o_loss = model(data)
        # if epoch<100:
        loss = F.nll_loss(y_out, data.y) + mc_loss + o_loss
        # else :
        #     loss = F.nll_loss(y_out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out, mc_loss, o_loss = model(data)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y)
        loss += float(F.nll_loss(y_out, data.y) * data.num_graphs + mc_loss + o_loss)
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    return (
        f1_score(y_true=y_true, y_pred=y_pred, average="macro"),
        accuracy_score(y_true=y_true, y_pred=y_pred),
        loss / len(loader.dataset),
    )

In [ ]:
best_macro_f1 = -1
train_losses = []
val_losses = []
train_accs = []
val_accs = []

In [ ]:
for epoch in range(400):
    train(epoch)
    train_macro_f1, train_acc, train_loss = test(train_loader)
    test_macro_f1, test_acc, test_loss = test(test_loader)
    # scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_f1: {train_macro_f1:01.4f},Test_f1: {test_macro_f1:01.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/quantile-MinCut_GCNConv.pth")
    train_losses.append(train_loss)
    val_losses.append(test_loss)
    train_accs.append(train_acc)
    val_accs.append(test_acc)
elapsed_time = time.time() - start

In [ ]:
utils.save_model_stats(
    "quantile-MinCut_GCNConv",
    train_losses,
    val_losses,
    train_accs,
    val_accs,
    elapsed_time,
)